<a href="https://colab.research.google.com/github/maxi74u/llm_diacritization_finetuning/blob/main/notebooks/finetuning_with_QLoRA" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers datasets accelerate peft bitsandbytes sentencepiece protobuf pyarabic torch huggingface_hub wandb # Optional for logging

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
bigframes 1

In [ ]:
import torch
from transformers import TrainingArguments
from peft import LoraConfig
from wandb import wandb
from huggingface_hub import HfApi, HfFolder
import pyarabic.araby
from datasets import load_dataset
from transformers import AutoTokenizer
import re
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, DataCollatorForLanguageModeling
from peft import prepare_model_for_kbit_training, get_peft_model

2025-05-11 14:08:41.943928: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746972522.124948      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746972522.177843      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
try:
  from kaggle_secrets import UserSecretsClient
  user_secrets = UserSecretsClient()
  secret_value_0 = user_secrets.get_secret("HF_TOKEN")
  secret_value_1 = user_secrets.get_secret("WANDB_TOKEN")
except Exception as e:
  print(e)

No module named 'kaggle_secrets'


In [ ]:
import sys

secret_value_0 = None
secret_value_1 = None

try:
    import os
    secret_value_0 = os.environ.get("HF_TOKEN")
    secret_value_1 = os.environ.get("WANDB_TOKEN")
    print("✅ Loaded secrets from environment variables (local).")
except Exception as e:
    print(f"❌ Failed to load secrets. Exception: {type(e).__name__}: {e}")

# Try importing kaggle_secrets for Kaggle
try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    secret_value_0 = user_secrets.get_secret("HF_TOKEN")
    secret_value_1 = user_secrets.get_secret("WANDB_TOKEN")
    print("✅ Loaded secrets from Kaggle.")
except ImportError:
    pass  # Proceed to other methods if not Kaggle

# Try getting secrets from Colab
try:
    from google.colab import userdata
    secret_value_0 = userdata.get("HF_TOKEN")
    secret_value_1 = userdata.get('WANDB_TOKEN')
    print("✅ Loaded secrets via Colab input.")
except ImportError:
    pass  # Proceed to other methods if not Colab

# Try getting secrets from environment variables for local use


# Check if secrets are loaded successfully
if not secret_value_0 or not secret_value_1:
    print("❌ Some secrets are missing.")


✅ Loaded secrets from environment variables (local).
✅ Loaded secrets via Colab input.


In [ ]:

# Your Hugging Face token as a string
token = secret_value_0

# Save the token so that other Hugging Face tools can use it
HfFolder.save_token(token)

# (Optional) If you want to use the API directly
api = HfApi(token=token)

# Your W&B API key
api_key = secret_value_1


# Log in programmatically
wandb.login(key=api_key)


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: bishertello (bishertello-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

# Configuration

In [ ]:
# Model ID
base_model_id = "HuggingFaceTB/SmolLM2-135M-Instruct"
# Dataset ID
dataset_id = "Bisher/fadel-diacritization"
# New model ID for saving adapters
new_model_id = f"{base_model_id.split('/')[1]}-fadel-full-arabic-diacritization" # Choose your preferred name

# QLoRA config
bnb_config = {
    "load_in_4bit": True,
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_compute_dtype": torch.bfloat16, # Use float16 if bfloat16 is not supported
    "bnb_4bit_use_double_quant": True,
}

# LoRA config
lora_config = LoraConfig(
    r=32, # Rank: Higher means more trainable parameters, adjust based on performance/memory
    lora_alpha=64, # Scaling factor (often 2*r)
    target_modules=[ # Modules to apply LoRA to - check model architecture if needed
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj"
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

# Training Arguments

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir=f"/kaggle/working/train_run-{new_model_id}",
    num_train_epochs=1, # Start with 1 epoch, increase if needed
    per_device_train_batch_size=8, # Adjust based on GPU memory
    gradient_accumulation_steps=2, # Increase effective batch size (4*4=16)
    gradient_checkpointing=True, # Saves memory
    optim="paged_adamw_32bit", # Optimizer recommended for QLoRA
    logging_steps=25,
    save_strategy="steps",
    save_steps=500, # Save checkpoints periodically
    eval_strategy="steps",
    eval_steps=500, # Evaluate periodically
    learning_rate=2e-4,
    # weight_decay=0.001,
    bf16=True if torch.cuda.is_bf16_supported() else False, # Use bf16 if available for faster training
    # tf32=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant", # Or "cosine"
    push_to_hub=True, # Set to True to push model adapters to Hub
    report_to="wandb", # Optional: set to "tensorboard" or "none"
    load_best_model_at_end=True, # Load the best model based on eval loss
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    label_names=["labels"], # <--- ADD THIS LINE
    run_name =f"{new_model_id}"
    # Consider adding max_seq_length if sequences are very long, but tokenizer should handle it
)


In [ ]:
training_args

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=True,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=500,
eval_strategy=IntervalStrategy.STEPS,
eval_use_gather_object=False,


# Prompt Format

In [ ]:

# --- Constants for Prompt Formatting ---
SYSTEM_PROMPT = "<|im_start|>system\nYou are an expert Arabic linguist. Your task is to add diacritics (Tashkeel) to the given Arabic text accurately.<|im_end|>"
USER_PROMPT_TEMPLATE = "<|im_start|>user\nDiacritize the following text:\n{undiacritized_text}<|im_end|>"
ASSISTANT_PROMPT_TEMPLATE = "<|im_start|>assistant\n{diacritized_text}<|im_end|>" # Note: includes EOS token

# Load dataset

In [ ]:
!ls '/kaggle/working/'

__notebook__.ipynb


In [ ]:
# --- Data Loading ---
import datasets
# dataset = load_dataset(dataset_id) # Load the training split
# dataset = load_dataset(dataset_id) # Load the training split
!cp -r '/kaggle/input/fadel-diacritization/fadel-diacritization-local' ./data
dataset = datasets.load_from_disk('./data')

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 128334
    })
    test: Dataset({
        features: ['text'],
        num_rows: 6767
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 6454
    })
})

In [ ]:
# --- Tokenizer Loading ---
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    padding_side="right", # Pad right for Causal LM
    trust_remote_code=True,
    use_fast=False # Qwen tokenizers might need use_fast=False
)
# Set pad token if not already set (Qwen uses <|endoftext|>)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print("Set pad_token to eos_token:", tokenizer.pad_token)

# --- Preprocessing Functions ---
def remove_diacritics(text):
    """Removes Arabic diacritics using pyarabic."""
    return pyarabic.araby.strip_tashkeel(text)

def format_prompt(example):
    """Creates the full instruction prompt for a given example."""
    diacritized = example['text']
    # Basic cleaning: Remove extra spaces, ensure consistency
    diacritized = re.sub(r'\s+', ' ', diacritized).strip()
    undiacritized = remove_diacritics(diacritized)

    if not undiacritized or not diacritized: # Skip empty examples if any
        return None

    user_prompt = USER_PROMPT_TEMPLATE.format(undiacritized_text=undiacritized)
    assistant_prompt = ASSISTANT_PROMPT_TEMPLATE.format(diacritized_text=diacritized)

    full_prompt = f"{SYSTEM_PROMPT}\n{user_prompt}\n{assistant_prompt}"
    return {"text": full_prompt, "input_text": user_prompt, "output_text": assistant_prompt}

def tokenize_function(examples):
    """Tokenizes the formatted prompts."""
    # Note: Qwen uses specific chat template logic, but here we format manually
    # We tokenize the full prompt. The loss will be calculated only on the assistant's part.
    tokenized_outputs = tokenizer(
        examples["text"],
        truncation=True,
        padding=False, # Trainer will handle padding with data collator
        max_length=512, # Adjust based on dataset analysis and memory
        add_special_tokens=False # We added special tokens manually in format_prompt
    )
    return tokenized_outputs

def create_labels(examples):
    """Creates labels for Causal LM training. Masks input tokens."""
    labels = []
    tokenizer_eos_token_id = tokenizer.eos_token_id # Store for quicker access

    for i in range(len(examples['input_ids'])):
        # Tokenize the input part (user prompt) and output part (assistant response) separately
        # to find where the assistant's response starts.
        # This requires knowing the exact format including special tokens.
        # Let's re-tokenize the input and output parts used to build the full prompt.
        # This is slightly inefficient but ensures accuracy.

        # Reconstruct the input section for tokenization
        prompt_input_part = f"{SYSTEM_PROMPT}\n{examples['input_text'][i]}\n<|im_start|>assistant\n"
        input_tokens = tokenizer(
            prompt_input_part,
            truncation=False, # Don't truncate here, we just need the length
            padding=False,
            add_special_tokens=False
        )["input_ids"]

        input_token_len = len(input_tokens)
        full_tokenized_len = len(examples['input_ids'][i])

        # Initialize labels with -100 (ignored by loss function)
        label = [-100] * full_tokenized_len

        # Unmask the tokens corresponding to the assistant's response
        # Start unmasking *after* the input part tokens
        for j in range(input_token_len, full_tokenized_len):
             # Check if it's not a pad token (though DataCollator handles this too)
            if examples['input_ids'][i][j] != tokenizer.pad_token_id:
                 label[j] = examples['input_ids'][i][j]

        # Ensure the EOS token at the very end is included in labels if present
        if examples['input_ids'][i][-1] == tokenizer_eos_token_id:
             if label[-1] == -100 : # Make sure it wasn't masked out by error
                 label[-1] = tokenizer_eos_token_id
        else:
             # If the full sequence was truncated before EOS, we might not have it.
             pass # Or add EOS manually if required by model/training setup


        labels.append(label)

    return {"labels": labels}


# --- Apply Preprocessing ---
print("Formatting prompts...")
formatted_dataset = dataset.map(format_prompt, remove_columns=['text'])
# Filter out None examples if any were created
formatted_dataset = formatted_dataset.filter(lambda example: example['text'] is not None)

print("Tokenizing data...")
# Keep original input/output text temporarily for label creation
tokenized_dataset = formatted_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=['text'] # Remove original combined text after tokenization
)

print("Creating labels...")
final_dataset = tokenized_dataset.map(
    create_labels,
    batched=True,
    remove_columns=['input_text', 'output_text'] # Remove helper columns
)

# Shuffle the datasets
train_dataset = final_dataset["train"]
eval_dataset = final_dataset["validation"]

print(f"Train dataset size: {len(train_dataset)}")
print(f"Eval dataset size: {len(eval_dataset)}")
print("Example processed sample:")
print(train_dataset[0])
print("\nDecoded Example:")
print(tokenizer.decode(train_dataset[0]['input_ids']))
print("\nLabels (first 100 tokens):")
print(train_dataset[0]['labels'][:100])

tokenizer_config.json:   0%|          | 0.00/3.76k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

...

Creating labels...


Map:   0%|          | 0/128334 [00:00<?, ? examples/s]

Map:   0%|          | 0/6767 [00:00<?, ? examples/s]

Map:   0%|          | 0/6454 [00:00<?, ? examples/s]

Train dataset size: 128334
Eval dataset size: 6454
Example processed sample:
{'input_ids': [1, 9690, 198, 2683, 359, 354, 4507, 12428, 8229, 363, 30, 2789, 3856, 314, 288, 803, 801, 387, 608, 747, 365, 68, 1317, 396, 299, 25, 288, 260, 1836, 12428, 1694, 8589, 30, 2, 198, 1, 4093, 198, 12117, 387, 608, 836, 260, 1695, 1694, 42, 198, 23939, 12667, 23939, 14787, 122, 21706, 34966, 14787, 121, 21706, 45704, 12667, 21706, 14787, 120, 20602, 48934, 14787, 126, 48934, 18223, 36395, 18223, 27819, 163, 113, 23939, 12667, 164, 227, 45726, 163, 132, 12667, 27438, 10805, 35964, 23339, 34966, 23339, 27749, 22682, 21706, 10805, 14787, 122, 10805, 21706, 34966, 10805, 14787, 230, 2, 198, 1, 520, 9531, 198, 23939, 14776, 12667, 14776, 23939, 29372, 14787, 122, 14776, 21706, 14776, 34966, 14776, 14787, 121, 30203, 21706, 38105, 14776, 45704, 14776, 12667, 25694, 21706, 14776, 14787, 120, 14776, 20602, 29372, 48934, 14776, 14787, 126, 30203, 48934, 29372, 18223, 164, 231, 36395, 25694, 18223, 29372, 27

# Load model

In [ ]:

# --- Load Base Model with Quantization ---
print("Loading base model...")
quantization_config = BitsAndBytesConfig(
    load_in_4bit=bnb_config["load_in_4bit"],
    bnb_4bit_quant_type=bnb_config["bnb_4bit_quant_type"],
    bnb_4bit_compute_dtype=bnb_config["bnb_4bit_compute_dtype"],
    bnb_4bit_use_double_quant=bnb_config["bnb_4bit_use_double_quant"],
)

model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=quantization_config,
    device_map="auto", # Automatically distribute across available GPUs
    trust_remote_code=True
)
print("Base model loaded.")

# --- Prepare Model for PEFT ---
# Enable gradient checkpointing for memory saving
model.gradient_checkpointing_enable()

# Prepare model for k-bit training (additional adjustments for quantization)
model = prepare_model_for_kbit_training(model)
print("Model prepared for k-bit training.")

# --- Apply LoRA ---
model = get_peft_model(model, lora_config)
print("PEFT model created.")

# --- Print Trainable Parameters ---
model.print_trainable_parameters()



Loading base model...


config.json:   0%|          | 0.00/861 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

Base model loaded.
Model prepared for k-bit training.
PEFT model created.
trainable params: 9,768,960 || all params: 144,283,968 || trainable%: 6.7706


In [ ]:
from transformers import DataCollatorForSeq2Seq # Make sure this import is added

# --- Data Collator ---
# Use DataCollatorForSeq2Seq which robustly handles padding for inputs and labels
print("Using DataCollatorForSeq2Seq.")
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    padding=True, # Ensure padding is enabled
    label_pad_token_id=-100 # Explicitly define the padding value for labels
)

Using DataCollatorForSeq2Seq.


# Train and push

In [ ]:
from transformers import Trainer
import os


In [ ]:
training_args.push_to_hub

True

In [ ]:


# Optional: Login to WandB
# wandb.login()

# --- Initialize Trainer ---
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    processing_class=tokenizer,
    data_collator=data_collator,

    # You could add compute_metrics=compute_metrics here for custom eval
)

# --- Start Training ---
print("Starting training...")
trainer.train()
print("Training finished.")

# --- Save the Final LoRA Adapter ---
print("Saving LoRA adapter...")
final_adapter_dir = os.path.join(training_args.output_dir, "final_adapter")
trainer.model.save_pretrained(final_adapter_dir)
tokenizer.save_pretrained(final_adapter_dir) # Save tokenizer with adapter
print(f"Adapter saved to {final_adapter_dir}")

trainer.push_to_hub()
print("Adapter pushed to Hugging Face Hub.") #

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Starting training...


wandb: Tracking run with wandb version 0.19.6
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250511_141440-vjcqv0hx
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run SmolLM2-135M-Instruct-fadel-full-arabic-diacritization
wandb: ⭐️ View project at https://wandb.ai/bishertello-/huggingface
wandb: 🚀 View run at https://wandb.ai/bishertello-/huggingface/runs/vjcqv0hx
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss,Validation Loss
500,0.134000,0.132819
1000,0.099900,0.102239
1500,0.081000,0.085592
2000,0.069100,0.076976
2500,0.066800,0.069491
3000,0.064200,0.065941
3500,0.060500,0.063670
4000,0.058600,0.058567
4500,0.053900,0.055704
5000,0.054500,0.054056


Training finished.
Saving LoRA adapter...
Adapter saved to /kaggle/working/train_run-SmolLM2-135M-Instruct-fadel-full-arabic-diacritization/final_adapter
Adapter pushed to Hugging Face Hub.


In [ ]:
pip show wandb

Name: wandb
Version: 0.19.6
Summary: A CLI and library for interacting with the Weights & Biases API.
Home-page: 
Author: 
Author-email: Weights & Biases <support@wandb.com>
License: MIT License

Copyright (c) 2021 Weights and Biases, Inc.

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEM

# Inference test

In [ ]:
from peft import PeftModel
import re

# --- Load Base Model and Adapter for Inference ---
print("Loading model for inference...")

# Load the base model with quantization (same config as training)
base_model_for_inference = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=quantization_config, # Use the same BNB config
    device_map="auto",
    trust_remote_code=True
)

# Load the PEFT adapter
adapter_path = final_adapter_dir # Use the path where the adapter was saved
inference_model = PeftModel.from_pretrained(base_model_for_inference, adapter_path)
inference_model.eval() # Set to evaluation mode

print("Model ready for inference.")

# --- Inference Function ---
def diacritize_text(text, model, tokenizer):
    """Takes undiacritized text and returns the model's diacritized version."""
    # Prepare the prompt
    undiacritized = re.sub(r'\s+', ' ', text).strip() # Basic cleaning
    user_prompt = USER_PROMPT_TEMPLATE.format(undiacritized_text=undiacritized)
    # Construct the prompt that the model expects *up to* the assistant's turn
    prompt = f"{SYSTEM_PROMPT}\n{user_prompt}\n<|im_start|>assistant\n"

    # Tokenize the input
    inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).to(model.device)

    # Generate output
    # Adjust max_new_tokens based on expected output length; add other generation params as needed
    # Using greedy decoding here, consider beam search, top-k, top-p for potentially better results
    outputs = model.generate(
        **inputs,
        max_new_tokens=len(inputs['input_ids'][0]) + 100, # Allow for diacritics and some buffer
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
        do_sample=False # Use greedy decoding for consistency
    )

    # Decode the generated tokens
    # Decode only the *newly generated* tokens
    generated_ids = outputs[0, inputs['input_ids'].shape[1]:]
    decoded_output = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()

    # Sometimes the model might hallucinate extra text after the main diacritization.
    # A simple heuristic: stop at the first newline or excessive length if needed.
    # decoded_output = decoded_output.split('\n')[0] # Simple cleanup if needed

    return decoded_output

# --- Example Usage ---
input_text = "اكل الولد التفاحة في الحديقة"
print(f"\nInput Text: {input_text}")

diacritized_output = diacritize_text(input_text, inference_model, tokenizer)
print(f"Model Output: {diacritized_output}")

input_text_2 = "بسم الله الرحمن الرحيم"
print(f"\nInput Text: {input_text_2}")
diacritized_output_2 = diacritize_text(input_text_2, inference_model, tokenizer)
print(f"Model Output: {diacritized_output_2}")

input_text_3 = "تعتبر اللغة العربية من أصعب اللغات في العالم"
print(f"\nInput Text: {input_text_3}")
diacritized_output_3 = diacritize_text(input_text_3, inference_model, tokenizer)
print(f"Model Output: {diacritized_output_3}")

Loading model for inference...
Model ready for inference.

Input Text: اكل الولد التفاحة في الحديقة
Model Output: اكْلَ الْوَلَدِ التُّفَاحَةَ فِي الْحَدِيقَةِ

Input Text: بسم الله الرحمن الرحيم
Model Output: بِسْمِ اللَّهِ الرَّحْمَنِ الرَّحِيمِ

Input Text: تعتبر اللغة العربية من أصعب اللغات في العالم
Model Output: تُعْتَبَرُ اللُّغَةُ الْعَرَبِيَّةُ مِنْ أَصْعَبِ اللُّغَاتِ فِي الْعَالِمِ
